In [1]:
import pandas as pd
import pickle
import preprocess_utils
import numpy as np
import json

from_database = False

if from_database:
    # Load tables from database 
    dfs = preprocess_utils.load_all_tables()
else:
    # Open the file in binary read mode and load the pickle data
    with open('preprocessed_data/dfs.pickle', 'rb') as f:
        dfs = pickle.load(f)
        for key, df in dfs.items():
            print(f"DataFrame {key:<20} has {len(df):<6} rows and {len(df.columns):<6} columns.")

if from_database:
    # Save tables to pickle
    with open('preprocessed_data/dfs.pickle', 'wb') as f:
        pickle.dump(dfs, f)

    # load it back with:
    with open('preprocessed_data/dfs.pickle', 'rb') as f:
        dfs = pickle.load(f)

DataFrame abuse_report_types   has 4      rows and 4      columns.
DataFrame abuse_reports        has 0      rows and 8      columns.
DataFrame ar_internal_metadata has 1      rows and 4      columns.
DataFrame articles             has 400    rows and 33     columns.
DataFrame assertion_types      has 10     rows and 11     columns.
DataFrame assertion_versions   has 19668  rows and 10     columns.
DataFrame assertions           has 13941  rows and 14     columns.
DataFrame assertions_genes     has 2053   rows and 2      columns.
DataFrame assertions_tags      has 403    rows and 2      columns.
DataFrame assertions_technique_types has 0      rows and 2      columns.
DataFrame assertions_techniques has 0      rows and 2      columns.
DataFrame assessment_types     has 13     rows and 9      columns.
DataFrame assessments          has 0      rows and 6      columns.
DataFrame authors              has 797    rows and 10     columns.
DataFrame banned_orcid_users   has 0      rows and 5   

## A. Database articles, claims and authors

### A.1. Process journal articles from the database

In [2]:
# Process articles
articles = preprocess_utils.clean_df_from_database(dfs["articles"])
articles = articles.rename(columns={"id": "article_id"})
# add journals to articles:
journals = preprocess_utils.clean_df_from_database(dfs["journals"])
journals = journals.drop('tag', axis=1).rename(columns={"id": "journal_id", "name": "journal_name"})
articles = articles.merge(journals, on="journal_id", how="left", suffixes=('', '_journal')).drop("journal_id", axis=1)


# Extract first and last authors from authors_txt
def extract_authors(authors_txt):
    if isinstance(authors_txt, str) and authors_txt:
        authors_list = authors_txt.split(';')
        first_author = authors_list[0].strip() if authors_list else None
        last_author = authors_list[-1].strip() if authors_list else None
        return pd.Series([first_author, last_author])
    return pd.Series([None, None])

# Apply the function to create new columns
articles[['first_author_extracted', 'last_author_extracted']] = articles['authors_txt'].apply(extract_authors)

# Check the results
print(f"Total articles: {len(articles)}")
print(f"Articles with first author: {articles['first_author_extracted'].notna().sum()}")
print(f"Articles with last author: {articles['last_author_extracted'].notna().sum()}")

articles = articles.drop(columns=["key", "references_txt", "additional_context", "num",  "nber_panels", "large_scale", "nber_tables", "published_at"])


# Display the first few rows to verify
articles[['authors_txt', 'first_author_extracted', 'last_author_extracted']].head()

Total articles: 400
Articles with first author: 400
Articles with last author: 400


,authors_txt,first_author_extracted,last_author_extracted
0,Rizki RM;Rizki TM,Rizki RM,Rizki TM
1,Leulier F;Lhocine N;Lemaitre B;Meier P,Leulier F,Meier P
2,Kim YS;Han SJ;Ryu JH;Choi KH;Hong YS;Chung YH;...,Kim YS,Lee WJ
3,Tingvall TO;Roos E;Engström Y,Tingvall TO,Engström Y
4,Bhaskar V;Valentine SA;Courey AJ,Bhaskar V,Courey AJ


In [3]:


# We need to analyze and clean up affiliations in the articles dataframe

def extract_primary_affiliation(affs_json_str):
    """Extract the first affiliation from the JSON string, safely handle bad JSON format."""
    try:
        # Parse the JSON string to a Python object
        affs_list = json.loads(affs_json_str)
        
        # Get the first non-empty affiliation
        for aff in affs_list:
            if aff and isinstance(aff, list) and len(aff) > 0 and aff[0].strip():
                return aff[0].strip()
        
        return None  # No valid affiliation found
    except (json.JSONDecodeError, TypeError, IndexError):
        # Handle poorly formatted JSON or other errors
        return None

# Create a new column with clean affiliations
articles['primary_affiliation'] = articles['affs_json'].apply(extract_primary_affiliation)

# Count articles with multiple vs. single affiliations
def count_affiliations(affs_json_str):
    """Count the number of non-empty affiliations in the JSON string."""
    try:
        affs_list = json.loads(affs_json_str)
        count = sum(1 for aff in affs_list if aff and isinstance(aff, list) and len(aff) > 0 and aff[0].strip())
        return count
    except (json.JSONDecodeError, TypeError, IndexError):
        return 0

articles['affiliation_count'] = articles['affs_json'].apply(count_affiliations)

# Identify articles with multiple affiliations
multiple_aff_articles = articles[articles['affiliation_count'] > 1]

# Print summary statistics
print(f"Total articles: {len(articles)}")
print(f"Articles with 1 affiliation: {len(articles[articles['affiliation_count'] == 1])}")
print(f"Articles with multiple affiliations: {len(multiple_aff_articles)}")
print(f"Articles with no valid affiliations: {len(articles[articles['affiliation_count'] == 0])}")

# Sample of articles with multiple affiliations
if not multiple_aff_articles.empty:
    print("\nSample of articles with multiple affiliations:")
    sample = multiple_aff_articles.head(5)
    for _, row in sample.iterrows():
        print(f"Article ID: {row['article_id']}, Title: {row['title'][:50]}...")
        try:
            affs = json.loads(row['affs_json'])
            for i, aff in enumerate(affs):
                if aff and isinstance(aff, list) and len(aff) > 0 and aff[0].strip():
                    print(f"  Affiliation {i+1}: {aff[0]}")
        except:
            print("  Error parsing affiliations")
        print()

Total articles: 400
Articles with 1 affiliation: 387
Articles with multiple affiliations: 0
Articles with no valid affiliations: 13


In [4]:
articles = articles.drop(columns=["affs_json", "affiliation_count"])

# Update impact factor for "Proceedings. Biological sciences" and standardize the journal name
articles.loc[articles["journal_name"] == "Proceedings. Biological sciences", "impact_factor"] = 4.7 
articles.loc[articles["journal_name"] == "Proceedings. Biological sciences", "journal_name"] = "Proceedings Biological Sciences"


In [5]:
articles = articles.apply(preprocess_utils.safe_strip)
articles.to_csv("preprocessed_data/articles_db.csv", index=False)

In [6]:
articles

,article_id,authors_txt,title,pmid,volume,issue,abstract,year,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation
0,3347,Rizki RM;Rizki TM,Cell interactions in the differentiation of a ...,111992,12,3,The cellular events in the formation of melano...,1979,Differentiation; research in biological diversity,3.39,Rizki RM,Rizki TM,None
1,1987,Leulier F;Lhocine N;Lemaitre B;Meier P,The Drosophila inhibitor of apoptosis protein ...,16894030,26,21,The founding member of the inhibitor of apopto...,2006,Molecular and cellular biology,5.30,Leulier F,Meier P,The Breakthrough Toby Robins Breast Cancer Res...
2,2761,Kim YS;Han SJ;Ryu JH;Choi KH;Hong YS;Chung YH;...,"Lipopolysaccharide-activated kinase, an essent...",10636911,275,3,Eukaryotic organisms use a similar Rel/NF-kapp...,2000,The Journal of biological chemistry,4.80,Kim YS,Lee WJ,"Laboratory of Immunology, Medical Research Cen..."
3,2649,Tingvall TO;Roos E;Engström Y,The GATA factor Serpent is required for the on...,11274409,98,7,Innate immunity in Drosophila is characterized...,2001,Proceedings of the National Academy of Science...,11.10,Tingvall TO,Engström Y,Department of Molecular Biology and Functional...
4,3441,Bhaskar V;Valentine SA;Courey AJ,A functional interaction between dorsal and co...,10660560,275,6,To identify proteins that regulate the functio...,2000,The Journal of biological chemistry,4.80,Bhaskar V,Courey AJ,"Department of Chemistry and Biochemistry, Univ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1882,Derré I;Pypaert M;Dautry-Varsat A;Agaisse H,RNAi screen in Drosophila cells reveals the in...,17967059,3,10,Chlamydia spp. are intracellular obligate bact...,2007,PLoS pathogens,6.70,Derré I,Agaisse H,"Section of Microbial Pathogenesis, Yale Univer..."
396,1815,Schmidt RL;Trejo TR;Plummer TB;Platt JL;Tang AH,Infection-induced proteolysis of PGRP-LC contr...,18308747,22,3,The Drosophila immune deficiency (IMD) pathway...,2008,FASEB journal : official publication of the Fe...,4.80,Schmidt RL,Tang AH,Department of Biochemistry and Molecular Biolo...
397,1884,Avet-Rochex A;Perrin J;Bergeret E;Fauvarque MO,Rac2 is a major actor of Drosophila resistance...,17903178,12,10,Pathogen recognition and engulfment by phagocy...,2007,Genes to cells : devoted to molecular & cellul...,2.10,Avet-Rochex A,Fauvarque MO,"Commissariat à l'Energie Atomique, DSV, iRTSV,..."
398,2041,Tanji T;Ohashi-Kobayashi A;Natori S,Participation of a galactose-specific C-type l...,16475980,396,1,A galactose-specific C-type lectin has been pu...,2006,The Biochemical journal,4.10,Tanji T,Natori S,"Department of Cell Biochemistry, Graduate Scho..."


### A.2. Process claims from the database

claims is merged with articles


In [7]:
# Main processing
claims = dfs["assertions"].copy()
print(len(claims))
claims = claims[claims["obsolete"] == False].copy()
print(len(claims))
claims = preprocess_utils.clean_df_from_database(claims)

id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)

claims = claims.merge(articles, on="article_id", how="left", suffixes=('', '_article'))
id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)

# Process assertion types
assertion_types = preprocess_utils.clean_df_from_database(dfs["assertion_types"])
assertion_types = assertion_types.rename(columns={"id": "assertion_type_id", "name": "assertion_type"})
claims = claims.merge(assertion_types, on="assertion_type_id", how="left", suffixes=('', '_assertion_type')).drop("assertion_type_id", axis=1)

# Process assessment types
assessment_types = preprocess_utils.clean_df_from_database(dfs["assessment_types"])
assessment_types = assessment_types.rename(columns={"id": "assessment_type_id", "name": "assessment_type"})
claims = claims.merge(assessment_types, on="assessment_type_id", how="left", suffixes=('', '_assessment_type')).drop("assessment_type_id", axis=1)

id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)

13941
13299
['article_id', 'assertion_type_id', 'assessment_type_id']
['article_id', 'assertion_type_id', 'assessment_type_id']
['article_id']


In [8]:
claims = claims.drop(['badge_tag_classes','description'], axis=1) # most not consistently used accross dataset
claims = claims.set_index('id')

In [9]:
claims = claims.apply(preprocess_utils.safe_strip)
claims.to_csv('preprocessed_data/claims_db.csv')

string_columns = claims.select_dtypes(include='object').columns.drop(["assessment_type", "journal_name"])
claims_truncated = claims.copy()
for col in string_columns:
    if col in claims_truncated.columns:
        claims_truncated[col] = claims_truncated[col].apply(lambda x: preprocess_utils.truncate_string(x))

# Save truncated dataframe
claims_truncated.to_csv('preprocessed_data/claims_db_truncated_for_llm.csv', index=False)
claims_truncated

,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,abstract,...,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation,assertion_type,label,is_assessed,assessment_type,rank_assessment_type
id,,,,,,,,,,,,,,,,,,,,,
5838,"<p>Belozerov VE, Lin...",2049.0,1,None,Zhuang ZH;Zhou Y;Yu ...,Regulation of Drosop...,16014325.0,18,4,The p38 mitogen-acti...,...,Cellular signalling,4.8,Zhuang ZH,Ge BX,Signal Transduction ...,reference,Reference,False,NaN,NaN
693,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,"Science (New York, N.Y.)",56.9,Choe KM,Anderson KV,Molecular Biology Pr...,assessment,Assessment,False,Not assessed,13.0
695,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,"Science (New York, N.Y.)",56.9,Choe KM,Anderson KV,Molecular Biology Pr...,assessment,Assessment,False,Not assessed,13.0
5840,Verified by many sub...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,Umeå Centre for Mole...,assessment,Assessment,False,Verified,1.0
5839,Activation of a cell...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,Umeå Centre for Mole...,main_claim,Main claim,True,Verified,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14468,The DIF antiserum us...,2898.0,1,None,Williams MJ;Rodrigue...,The 18-wheeler mutat...,9321392.0,16,20,Mammals and insects ...,...,The EMBO journal,11.4,Williams MJ,Eldon ED,Department of Biolog...,comment,Comment,False,Not assessed,13.0
11330,Although independent...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,Journal of cell science,4.0,Shia AK,Ligoxygakis P,Department of Bioche...,assessment,Assessment,False,"Unchallenged, logically consistent",6.0
11329,Spatzle (#gene:FBgn0...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,Journal of cell science,4.0,Shia AK,Ligoxygakis P,Department of Bioche...,main_claim,Main claim,True,"Unchallenged, logically consistent",6.0


In [10]:
claims_truncated

,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,abstract,...,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation,assertion_type,label,is_assessed,assessment_type,rank_assessment_type
id,,,,,,,,,,,,,,,,,,,,,
5838,"<p>Belozerov VE, Lin...",2049.0,1,None,Zhuang ZH;Zhou Y;Yu ...,Regulation of Drosop...,16014325.0,18,4,The p38 mitogen-acti...,...,Cellular signalling,4.8,Zhuang ZH,Ge BX,Signal Transduction ...,reference,Reference,False,NaN,NaN
693,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,"Science (New York, N.Y.)",56.9,Choe KM,Anderson KV,Molecular Biology Pr...,assessment,Assessment,False,Not assessed,13.0
695,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,"Science (New York, N.Y.)",56.9,Choe KM,Anderson KV,Molecular Biology Pr...,assessment,Assessment,False,Not assessed,13.0
5840,Verified by many sub...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,Umeå Centre for Mole...,assessment,Assessment,False,Verified,1.0
5839,Activation of a cell...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,Umeå Centre for Mole...,main_claim,Main claim,True,Verified,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14468,The DIF antiserum us...,2898.0,1,None,Williams MJ;Rodrigue...,The 18-wheeler mutat...,9321392.0,16,20,Mammals and insects ...,...,The EMBO journal,11.4,Williams MJ,Eldon ED,Department of Biolog...,comment,Comment,False,Not assessed,13.0
11330,Although independent...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,Journal of cell science,4.0,Shia AK,Ligoxygakis P,Department of Bioche...,assessment,Assessment,False,"Unchallenged, logically consistent",6.0
11329,Spatzle (#gene:FBgn0...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,Journal of cell science,4.0,Shia AK,Ligoxygakis P,Department of Bioche...,main_claim,Main claim,True,"Unchallenged, logically consistent",6.0


### A.3. Process authors from database

there are about two author per article... though the total of authors is 797 so some are missing.

In [11]:
a = preprocess_utils.clean_df_from_database(dfs["authors"])  
a[a["article_id"] == 2053]

,id,article_id,name,sex,career_stage_id,leading_author,first_author,expertise_level_id
0,2,2053,Brun S,1,3,False,True,2
1,3,2053,Lemaitre B,1,4,True,False,2


In [12]:
len(a)

797

In [13]:
a[a["name"] == "Lemaitre B"].head()

,id,article_id,name,sex,career_stage_id,leading_author,first_author,expertise_level_id
1,3,2053,Lemaitre B,1,4,True,False,2
7,8,3437,Lemaitre B,1,4,True,False,2
14,16,2801,Lemaitre B,1,4,True,False,2
30,36,2617,Lemaitre B,1,4,True,False,2
44,56,2869,Lemaitre B,1,4,True,False,2


In [14]:
a[a["article_id"] == 2451]

,id,article_id,name,sex,career_stage_id,leading_author,first_author,expertise_level_id
17,19,2451,De Gregorio E,1,3,False,True,2
95,20,2451,Lemaitre B,1,5,False,False,2
789,795,2451,Brey PT,1,5,True,False,2


In [15]:
authors = preprocess_utils.clean_df_from_database(dfs["authors"].copy())
authors
career_stage = dfs["career_stages"].rename(columns={"id": "career_stage_id"})
authors = authors.merge(career_stage, on="career_stage_id", how="left", suffixes=('', '_career_stage')).drop("career_stage_id", axis=1)
expertise_level = dfs["expertise_levels"].rename(columns={"id": "expertise_level_id"})
authors = authors.merge(expertise_level, on="expertise_level_id", how="left", suffixes=('', '_expertise_level')).drop("expertise_level_id", axis=1)
authors = authors.rename(columns={"name_career_stage": "career_stage", "name_expertise_level": "expertise_level", "id": "author_id"})

# Convert sex values: 1 to "Male", 0 to "Female"
authors.loc[authors["sex"] == 1, "sex"] = "Male"
authors.loc[authors["sex"] == 0, "sex"] = "Female"
authors["author_key"] =  authors["name"].str.lower().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

authors

/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_25753/706437459.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Male' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  authors.loc[authors["sex"] == 1, "sex"] = "Male"


,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
0,2,2053,Brun S,Male,False,True,Post-doc,Experienced,brun s
1,3,2053,Lemaitre B,Male,True,False,Junior PI,Experienced,lemaitre b
2,1,1650,Charroux B,Male,False,True,Post-doc,Experienced,charroux b
3,97,2149,Söderhäll K,Male,True,False,Senior PI,Experienced,soderhall k
4,5,2278,Silverman N,Male,False,True,Post-doc,Newcomer,silverman n
...,...,...,...,...,...,...,...,...,...
792,797,2067,Cerenius L,Male,True,False,Senior PI,Experienced,cerenius l
793,798,2122,Higgins DE,Male,True,False,Senior PI,Newcomer,higgins de
794,799,2184,Mengin-Lecreulx D,Male,True,False,Senior PI,Experienced,mengin-lecreulx d
795,800,2130,Moore KJ,Female,True,False,Junior PI,Newcomer,moore kj


In [16]:
# check authors thare not both leading and first authors, or that are neither using XOR
authors[~(authors["leading_author"] ^ authors["first_author"])]

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
95,20,2451,Lemaitre B,Male,False,False,Senior PI,Experienced,lemaitre b
194,127,2113,Schneider DS,Male,False,False,Junior PI,Experienced,schneider ds
195,197,3440,Lemaitre B,Male,False,False,Junior PI,Experienced,lemaitre b
196,169,2184,Lemaitre B,Male,False,False,Junior PI,Experienced,lemaitre b
243,250,1947,Schneider DS,Male,True,True,Senior PI,Experienced,schneider ds
256,260,1763,Schneider DS,Male,False,False,Senior PI,Experienced,schneider ds
257,252,2730,Shahabuddin M,Male,False,False,Junior PI,Newcomer,shahabuddin m
287,303,3458,Hultmark D,Male,True,True,Senior PI,Experienced,hultmark d
329,251,2730,Schneider DS,Male,True,True,Junior PI,Newcomer,schneider ds
334,378,1702,Williams MJ,Male,True,True,Junior PI,Experienced,williams mj


In [17]:
#authors = authors.merge(articles, on="article_id", how="left", suffixes=('', '_article'))
#authors.sort_values(by=["article_id", "author_id", ])

authors = preprocess_utils.clean_author_keys(authors, "author_key")
authors = authors.apply(preprocess_utils.safe_strip)
authors.to_csv('preprocessed_data/authors_db.csv', index=False)

In [18]:
authors.sort_values(by=["author_key"])

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
724,745,1535,Abdelsadik A,Male,False,True,PhD,Experienced,abdelsadik a
485,496,2122,Agaisse H,Male,False,True,Post-doc,Experienced,agaisse h
487,499,1882,Agaisse H,Male,True,False,Junior PI,Experienced,agaisse h
15,17,2297,Agaisse H,Male,False,True,Post-doc,Experienced,agaisse h
104,108,1752,Aggarwal K,Female,False,True,PhD,Experienced,aggarwal k
...,...,...,...,...,...,...,...,...,...
754,784,1491,Zhu S,Male,True,False,Junior PI,Newcomer,zhu s
740,714,1756,Zhu S,Male,True,False,Junior PI,Newcomer,zhu s
653,621,1748,Zhu S,Male,True,False,Senior PI,Newcomer,zhu s
588,588,2025,Zhuang ZH,Male,False,True,PhD,Newcomer,zhuang zh


## B. Process from the XLSX files

- **3:citations** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/3_2025_March 9th_citation_counts.xlsm`
  -  il s’agit des citations : je ne l’ai pas travaillé.  
 - **4:first&last** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/4_working first and last.xlsx`
    -   ils’agit d’une liste de 20 auteurs qui sont first and last and mais ne pas prendre en compte ceux qui sont PI.  

 It seems that
 - sex -> FH
 - PhD Post-doc -> FH
 - Become a Pi -> FH
 - current job -> FH
 - MD -> FH
 - Affiliation -> claim
 - Country -> claim
 - Ivy League -> claim

### B.1. Read the Triage last_author files which contains all pairs

update April 7, 2025: we don't care avout this file anymore

In [ ]:
# Read Author info, which contains all the pairs
paper_auth_pairs = pd.read_excel('input_data/2025-02-14_last_xlsx/1_Triage_Last author.xlsx', sheet_name='Tri sans les doublons')
# Drop all columns with 'Unnamed' in the name
paper_auth_pairs = paper_auth_pairs.drop(columns=[col for col in paper_auth_pairs.columns if 'Unnamed' in col]).drop(columns=['Source'])
# rename RIZKI MT	to Rizki TM in column last author
paper_auth_pairs['last author'] = paper_auth_pairs['last author'].replace({'Rizki MT': 'Rizki TM','RIZKI MT': 'Rizki TM' })

paper_auth_pairs.loc[paper_auth_pairs["MD"] == "??", "MD"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["MD"] == "?", "MD"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 'O', "MD"] = False
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 0, "MD"] = False
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 1, "MD"] = True

paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == "??", "Become a Pi"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == 0, "Become a Pi"] = False
paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == 1, "Become a Pi"] = True

paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 0, "Ivy league"] = False
paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 1, "Ivy league"] = True

paper_auth_pairs.loc[paper_auth_pairs["Sex"] == "??", "Sex"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["Sex"] == 1, "Sex"] = "Male"
paper_auth_pairs.loc[paper_auth_pairs["Sex"] == 0, "Sex"] = "Female"

paper_auth_pairs.loc[paper_auth_pairs["current job"] == "pi", "current job"] = "PI"
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "Pi", "current job"] = "PI"
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "?", "current job"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "??", "current job"] = np.nan

# create merge columns: lowercased and stripped of accents
paper_auth_pairs['first_author_key'] = paper_auth_pairs['first author'].str.lower()
paper_auth_pairs['first_author_key'] = paper_auth_pairs['first_author_key'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
paper_auth_pairs = preprocess_utils.clean_author_keys(paper_auth_pairs, "first_author_key")

paper_auth_pairs.loc[:, 'last_author_key'] = paper_auth_pairs['last author'].str.lower()
paper_auth_pairs.loc[:, 'last_author_key'] = (paper_auth_pairs['last_author_key']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'))
#paper_auth_pairs.loc[:, 'last_author_key'] = paper_auth_pairs['last_author_key'].str.replace('rizki mt', 'rizki tm')
paper_auth_pairs = preprocess_utils.clean_author_keys(paper_auth_pairs, "last_author_key")


print(paper_auth_pairs["Sex"].unique(), paper_auth_pairs["MD"].unique(), paper_auth_pairs["Become a Pi"].unique(), paper_auth_pairs["Ivy league"].unique())

paper_auth_pairs = paper_auth_pairs.apply(preprocess_utils.safe_strip())


['Female' 'Male' nan] [False nan True] [True False nan] [True False]


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_25753/687267990.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 0, "Ivy league"] = False


#### Merge it with database to obtain the article ID

In [55]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
0,Agaisse H,Derré,Female,Post-doc,True,PI,False,Yale University,USA,True,derre i,agaisse h
1,Aguilera RJ,Seong CS,Male,PhD,False,Academia,False,The University of Texas,USA,False,seong cs,aguilera rj
2,Aigaki T,Tsuda M,Male,PhD,True,Admin,False,Tokyo Metropolitan University,Japan,False,tsuda m,aigaki t
3,Ando,Markus,Male,PhD,False,Facility leader,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,markus r,ando i
4,Ando,Rus F,Female,PhD,False,NaN,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,rus f,ando i
...,...,...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
316,Zhu S,Yuan Y,Male,PhD,False,Senior Staff,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,yuan y,zhu s
317,Zhu S,Tian C,Female,PhD,True,PI,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,tian c,zhu s
318,Zhu S,Zhang Z,Female,PhD,False,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhang z,zhu s


In [ ]:
print(len(authors.article_id.unique()))
authors

400


,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
0,2,2053,Brun S,Male,False,True,Post-doc,Experienced,brun s
1,3,2053,Lemaitre B,Male,True,False,Junior PI,Experienced,lemaitre b
2,1,1650,Charroux B,Male,False,True,Post-doc,Experienced,charroux b
3,97,2149,Söderhäll K,Male,True,False,Senior PI,Experienced,soderhall k
4,5,2278,Silverman N,Male,False,True,Post-doc,Newcomer,silverman n
...,...,...,...,...,...,...,...,...,...
792,797,2067,Cerenius L,Male,True,False,Senior PI,Experienced,cerenius l
793,798,2122,Higgins DE,Male,True,False,Senior PI,Newcomer,higgins de
794,799,2184,Mengin-Lecreulx D,Male,True,False,Senior PI,Experienced,mengin-lecreulx d
795,800,2130,Moore KJ,Female,True,False,Junior PI,Newcomer,moore kj


In [53]:
paper_auth_pairs["article_id"] = pd.NA
paper_auth_pairs["article_id"] = paper_auth_pairs["article_id"].astype("Int64")

In [ ]:
for art in authors["article_id"].unique():
    these_authors = authors[authors["article_id"] == art]
    these_first_authors = these_authors[these_authors["first_author"] == True]["author_key"].values
    these_leading_authors = these_authors[these_authors["leading_author"] == True]["author_key"].values
    if (len(these_first_authors) == 1 and len(these_leading_authors) == 1):
        indexes = paper_auth_pairs[(paper_auth_pairs["last_author_key"] == these_leading_authors[0]) & 
                                (paper_auth_pairs["first_author_key"] == these_first_authors[0])].index.to_list()
        if len(indexes) == 1:
            paper_auth_pairs.loc[indexes[0], "article_id"] = art
        elif len(indexes) > 1:
            first_na = pd.isna(paper_auth_pairs.loc[indexes, "article_id"]).idxmax() #idmax returns the index of the first occurrence of the maximum value
            print(these_authors["author_key"].values, pd.isna(paper_auth_pairs.loc[indexes, "article_id"]).values, first_na)
            paper_auth_pairs.loc[first_na, "article_id"] = art
        else:
            print(f"NO MATCH 🛑 {art}, {len(indexes)}, {these_first_authors[0]:<20} -- {these_leading_authors[0]:<20} ... {len(these_authors)}")
    else:
        print(f"SKIPPING {art} because only one={len(these_authors)} author: {these_authors['author_key'].values}")

['georgel p' 'hoffmann ja'] [ True  True] 96
NO MATCH 🛑 2239, 0, hedengren-olcott m   -- taylor bj            ... 2
['georgel p' 'hoffmann ja'] [False  True] 101
NO MATCH 🛑 1947, 0, schneider ds         -- schneider ds         ... 1
NO MATCH 🛑 2730, 0, schneider ds         -- schneider ds         ... 2
NO MATCH 🛑 3447, 0, rizki tm             -- rizki tm             ... 1
NO MATCH 🛑 3448, 0, rizki tm             -- rizki tm             ... 1
NO MATCH 🛑 3292, 0, rizki tm             -- rizki tm             ... 1
NO MATCH 🛑 3273, 0, rizki tm             -- rizki tm             ... 2
['georgel p' 'hoffmann ja'] [False False] 96
NO MATCH 🛑 2764, 0, imler jl             -- imler jl             ... 1
NO MATCH 🛑 3446, 0, nappi aj             -- nappi aj             ... 1


### There is an error above, : 3 hoffman georgel, whereas in the excel there are two...

In [ ]:
paper_auth_pairs[paper_auth_pairs["article_id"].isna()]

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key,article_id
16,Bellotti RA,Rizki TM,Male,PI,True,Deceased,False,Ann arbor,USA,False,rizki tm,bellotti ra,<NA>
106,Hoffmann JA,Imler JL,Male,PI,True,PI,False,CNRS-University of Strasbourg,France,False,imler jl,hoffmann ja,<NA>
240,Rizki TM,RIZKI MT,Male,PI,True,Retired,False,ann arbor,NaN,False,rizki mt,rizki tm,<NA>
271,Shahabuddin M,Schneider D,Male,Pi,True,PI,True,Whitehead Institute,USA,False,schneider d,shahabuddin m,<NA>
272,Shirasu-Hiza MM,Schneider DS,Male,Pi,True,PI,True,Stanford University Stanford,USA,True,schneider ds,shirasu-hiza mm,<NA>
278,Silvers MJ,Nappi AJ,Male,PI,True,PI,False,Univeristy of Chicago,USA,False,nappi aj,silvers mj,<NA>
290,Taylor BJ,Hedengren M,Female,PhD,False,NaN,False,Oregon State University,USA,False,hedengren m,taylor bj,<NA>


In [ ]:
paper_auth_pairs.loc[290, "article_id"] = 2239

In [ ]:
paper_auth_pairs[paper_auth_pairs["article_id"].isna()]

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key,article_id
16,Bellotti RA,Rizki TM,Male,PI,True,Deceased,False,Ann arbor,USA,False,rizki tm,bellotti ra,<NA>
106,Hoffmann JA,Imler JL,Male,PI,True,PI,False,CNRS-University of Strasbourg,France,False,imler jl,hoffmann ja,<NA>
240,Rizki TM,RIZKI MT,Male,PI,True,Retired,False,ann arbor,NaN,False,rizki mt,rizki tm,<NA>
271,Shahabuddin M,Schneider D,Male,Pi,True,PI,True,Whitehead Institute,USA,False,schneider d,shahabuddin m,<NA>
272,Shirasu-Hiza MM,Schneider DS,Male,Pi,True,PI,True,Stanford University Stanford,USA,True,schneider ds,shirasu-hiza mm,<NA>
278,Silvers MJ,Nappi AJ,Male,PI,True,PI,False,Univeristy of Chicago,USA,False,nappi aj,silvers mj,<NA>


In [ ]:
len(paper_auth_pairs.article_id.unique())

315

### B.2. Read the stat author files, both the sheet for first and last.
- **0: First** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/0_First author cleaned_March 18th.xlsx`
  - Pour les statistiques sur ceux qui deviennent PI, Il ne faut pas prendre en compte ceux qui sont surlignés en bleu car ils sont des premiers auteurs qui sont déjà PI. Normalement ce fichier est bon.  
Avec un peu d’approximation, je trouve que sur 291 first authors: 107 become PI et il y  16  ?? (indéterminés)et 13 qui sont déjà PI en bleu :  ce qui fait 107/262=>40,8% (262=291-16-13)  
- **2: Both** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx`
  - obtenu depuis le site
  -  tu trouveras les last avec de nouveau critères  pour historical (tradition), continuty, first and last.  (pour junior et senior il faut voir par articles dans la database)  
  - Feuille 1 : Parmi les first author (1er feuille) :  
    - Zhenting Zhang et Zhaolin Zhang  sont fusionnés en Zhang Z  
    - Zhipeng Wang et Zhi Wang sont fusionné en Wang Z  
    - Hedengren M et Hedengren M et Hedengren-Olcott M ne sont pas fusionnés
  - Feuille 3: pour les last authors:
    - Il faut fusionner Schneider DS et Schneider D -> c'est les mêmes
    - Historical lab (tradition) or not : il faut pendre le colonne Historical lab after 1998 et comparer 1 (historical, trained in hustorical lab) et 0 (non historical) mais ne pas prendre en compte 2.
    - La colonne E first and Last : compte les chercheurs qui apparaissent comme first author  et last authors. Il faudrait comparer 1 (last author qui ont été first author dans une autre labo mais pas PI)  à  0 (last author withouth being first author)  sans prendre en compte les 2 (first author who were already PI when first author).
  

In [ ]:
# read an xlx file in pandas
stat_author_fn = "input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx"

first_authors_claims = pd.read_excel(stat_author_fn, sheet_name='First')
leading_authors_claims = pd.read_excel(stat_author_fn, sheet_name='Leading')
leading_authors_claims["Authorship"]= "Leading"
first_authors_claims["Authorship"]= "First"

def clean_authors_claims(df):
    df['Sex'] = df['Sex'].map({1: 'Male', 0: 'Female', '??': np.nan, '?': np.nan})
    df = df.drop(columns=[col for col in df.columns if '%' in col])
    df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
    df = df.rename(columns={'Conituinity': 'Continuity', 
                                                "Partially verified":"Partially Verified", 
                                                "*Historical lab after 1998":"Historical lab after 1998"
                                                })
    if "Historical lab" in df.columns:
        df['Historical lab'] = df['Historical lab'].astype('boolean')
    if "Continuity" in df.columns:
        df['Continuity'] = df['Continuity'].astype('boolean')
    if "Become a Pi" in df.columns:
        df.loc[df["Become a Pi"] == "??", "Become a Pi"] = np.nan
        df.loc[df["Become a Pi"] == 0, "Become a Pi"] = False
        df.loc[df["Become a Pi"] == 1, "Become a Pi"] = True
    if "MD" in df.columns:
        df.loc[df["MD"] == "??", "MD"] = np.nan
        df.loc[df["MD"] == "?", "MD"] = np.nan
        df.loc[df["MD"] == 'O', "MD"] = False
        df.loc[df["MD"] == 0, "MD"] = False
        df.loc[df["MD"] == 1, "MD"] = True
    if "Ivy league" in df.columns:
        df.loc[df["Ivy league"] == 0, "Ivy league"] = False
        df.loc[df["Ivy league"] == 1, "Ivy league"] = True

    df = df.dropna(subset=['Name'])
    # drop  Schneider DS+D from names as  Schneider DS and  Schneider D are also there in separate rows
    df = df[~(df['Name'] == 'Schneider DS+D')]
    return df


first_authors_claims = clean_authors_claims(first_authors_claims)
leading_authors_claims = clean_authors_claims(leading_authors_claims)

leading_authors_claims = preprocess_utils.build_author_key(leading_authors_claims, author_name_col="Name", key_col="last_author_key")
first_authors_claims = preprocess_utils.build_author_key(first_authors_claims, author_name_col="Name", key_col="first_author_key")

In [5]:
first_authors_claims

,Name,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Authorship,first_author_key
0,Williams MJ,Male,5,14,7,3,1,0,3,First,williams mj
1,Lemaitre B,Male,4,12,0,11,1,0,0,First,lemaitre b
2,Ha EM,Female,4,10,3,1,2,2,2,First,ha em
3,Leulier F,Male,4,10,0,10,0,0,0,First,leulier f
4,De Gregorio E,Male,3,9,1,8,0,0,0,First,de gregorio e
...,...,...,...,...,...,...,...,...,...,...,...
287,Schlenke TA,Male,1,1,0,1,0,0,0,First,schlenke ta
288,Uttenweiler-Joseph S,Female,1,1,0,1,0,0,0,First,uttenweiler-joseph s
289,Wang LN,Female,1,1,0,1,0,0,0,First,wang ln
290,Yuan Y,Male,1,1,0,1,0,0,0,First,yuan y


In [8]:
first_authors_claims2_fn = "input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/0_First author cleaned_March 18th.xlsx"
first_authors_claims2 = pd.read_excel(first_authors_claims2_fn)
first_authors_claims2["Name"] = first_authors_claims2["AAA_first author"]
first_authors_claims2["Become a Pi"] = first_authors_claims2["AAA_Become a Pi"]
first_authors_claims2 = first_authors_claims2.drop(columns=["AAA_first author", "AAA_Become a Pi","Source"])

first_authors_claims2 = clean_authors_claims(first_authors_claims2)
print(first_authors_claims2["Sex"].unique(), 
        first_authors_claims2["MD"].unique(), 
        first_authors_claims2["Become a Pi"].unique(), 
        first_authors_claims2["Ivy league"].unique())
first_authors_claims2 = preprocess_utils.build_author_key(first_authors_claims2, author_name_col="Name", key_col="first_author_key")
first_authors_claims2

['Male' 'Female' nan] [False nan True] [True False nan] [False True]


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_41023/3466221200.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["Ivy league"] == 0, "Ivy league"] = False


,Sex,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key
0,Male,Senior Staff,PI,False,Forschungszentrum Borstel,Germany,False,Abdelsadik A,True,abdelsadik a
1,Male,Post-doc,PI,False,Harvard Medical School,USA,True,Agaisse H,True,agaisse h
2,Female,PhD,Industry,False,University of Massachusetts Medical School,USA,False,Aggarwal K,False,aggarwal k
3,NaN,PhD,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,Ao J,NaN,ao j
4,Male,Post-doc,PI,False,Harvard Medical School and Massachusetts Gener...,USA,True,Apidianakis Y,True,apidianakis y
...,...,...,...,...,...,...,...,...,...,...
285,Female,PhD,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,Zhang Z,False,zhang z
286,Female,PhD,Admin,False,University of Kentucky,USA,False,Zhang Z,False,zhang z
287,Female,PhD,??,False,University of California San Diego,USA,True,Zhao HW,False,zhao hw
288,Male,PhD,??,False,University of Houston,USA,False,Zhou Z,NaN,zhou z


### B.3 Merge to create a covariate file for Last Author

In [ ]:
paper_auth_pairs_LH = paper_auth_pairs[["last author", "last_author_key", "Affiliation", "Country", "Ivy league"]]
paper_auth_pairs_LH = preprocess_utils.deduplicate_by(paper_auth_pairs_LH, "last author").copy()

In [24]:
 # check for duplicate last_author_key in paper_auth_pairs_LH
paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('last_author_key', keep=False)]
print(len(paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('last_author_key', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate last_author_key in leading_authors_claims
print(len(leading_authors_claims[leading_authors_claims.duplicated('last_author_key', keep=False)]), "duplicates in leading_authors_claims")

0 duplicates in paper_auth_pairs_LH
0 duplicates in leading_authors_claims


In [25]:
# Perform the outer merge to see what we are missing
all_LH = pd.merge(leading_authors_claims, paper_auth_pairs_LH, on='last_author_key', how='outer')
print(len(leading_authors_claims), len(paper_auth_pairs_LH), len(all_LH))

157 160 161


In [26]:
unique_pairs = all_LH[["Name", "last author", "last_author_key"]].drop_duplicates().sort_values("last_author_key", ascending=True)
print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['last author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f"{unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['last_author_key']:<20}")
    #else:
    #    print(f"   {unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['last_author_key']:<20}")

 ONLY in Pairs               only in Claims        matching key 
----------------------------------------------------------------------
💥 Bellotti RA          vs  nan                  --> bellotti ra         
💥 nan                  vs  Nappi AJ             --> nappi aj            
💥 Shahabuddin M        vs  nan                  --> shahabuddin m       
💥 Shirasu-Hiza MM      vs  nan                  --> shirasu-hiza mm     
💥 Silvers MJ           vs  nan                  --> silvers mj          


In [27]:
all_LH_inner = pd.merge(leading_authors_claims, paper_auth_pairs_LH, on='last_author_key', how='inner').sort_values("last_author_key", ascending=True)
#all_LH_inner.drop(columns=['last_author_key', 'last author', 'Authorship'], inplace=True)
print(len(all_LH_inner))


assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']
# set type of assessment columns to int
for col in assessment_columns:
    all_LH_inner[col] = all_LH_inner[col].astype(int)
all_LH_inner['Major claims'] = all_LH_inner['Major claims'].astype(int)
all_LH_inner['Articles'] = all_LH_inner['Articles'].astype(int)

for col in assessment_columns:
    all_LH_inner[f'{col}_prop'] = all_LH_inner[col] / all_LH_inner['Major claims']

all_LH_inner = all_LH_inner.apply(preprocess_utils.safe_strip)
all_LH_inner.to_csv('preprocessed_data/LH_inner.csv', index=False)
all_LH_inner

156


,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Articles,Major claims,Unchallenged,Verified,...,last_author_key,last author,Affiliation,Country,Ivy league,Unchallenged_prop,Verified_prop,Partially Verified_prop,Mixed_prop,Challenged_prop
63,Agaisse H,False,0.0,False,1.0,Male,1,3,2,1,...,agaisse h,Agaisse H,Yale University,USA,True,0.666667,0.333333,0.000000,0.0,0.0
107,Aguilera RJ,False,0.0,False,0.0,Male,1,2,0,1,...,aguilera rj,Aguilera RJ,The University of Texas,USA,False,0.000000,0.500000,0.000000,0.0,0.5
64,Aigaki T,False,0.0,False,0.0,Male,1,3,2,1,...,aigaki t,Aigaki T,Tokyo Metropolitan University,Japan,False,0.666667,0.333333,0.000000,0.0,0.0
6,Anderson KV,False,0.0,False,0.0,Female,8,20,7,11,...,anderson kv,Anderson KV,Memorial Sloan-Kettering Cancer Center and the...,USA,True,0.350000,0.550000,0.000000,0.0,0.1
16,Andó I,False,0.0,True,0.0,Male,6,12,1,7,...,ando i,Ando,"Hungarian Academy of Sciences, Szeged",Hungary,False,0.083333,0.583333,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Xu T,False,0.0,False,0.0,Male,1,3,0,3,...,xu t,Xu T,Yale University School of Medicine,USA,False,0.000000,1.000000,0.000000,0.0,0.0
146,Yamaguchi M,False,0.0,False,0.0,Male,1,2,1,1,...,yamaguchi m,Yamaguchi M,Aichi Cancer Center Research Institute,Japan,False,0.500000,0.500000,0.000000,0.0,0.0
105,Yoo MA,False,0.0,False,0.0,Female,1,3,0,2,...,yoo ma,Yoo MA,Pusan National University,Korea,False,0.000000,0.666667,0.333333,0.0,0.0
106,Yu XQ,False,0.0,False,0.0,Male,1,3,1,2,...,yu xq,Yu XQ,niversity of Missouri-Kansas City,USA,False,0.333333,0.666667,0.000000,0.0,0.0


In [28]:
for aut in authors[authors["leading_author"]]["author_key"]:
    if len(all_LH[all_LH["last_author_key"] == aut]) != 1:
        print (f"outer: 💥 {aut}, {len(all_LH[all_LH['last_author_key'] == aut])}")
    if len(all_LH_inner[all_LH_inner["last_author_key"] == aut]) != 1:
        print (f"inner: 💥 {aut}, {len(all_LH_inner[all_LH_inner['last_author_key'] == aut])}")


inner: 💥 nappi aj, 0


### B. First authors

In [ ]:
all_FH = pd.merge(first_authors_claims, first_authors_claims2, on='first_author_key', how='outer', suffixes=('_claims', '_claims2'))
all_LH_inner = pd.merge(first_authors_claims, first_authors_claims2, on='first_author_key', how='inner').sort_values("first_author_key", ascending=True)
print(len(first_authors_claims), len(first_authors_claims2), len(all_FH), len(all_LH_inner))

292 290 295 291


In [18]:
first_authors_claims2.Affiliation.value_counts()

Affiliation
CNRS-University of Strasbourg                  32
University of Stockholm                        29
CNRS_Gif-sur-Yvette_Université Paris-Saclay    20
University of Massachusetts Medical School     15
CNRS University of Marseille                    7
                                               ..
University of Uppsala                           1
Kansas State University                         1
University of Modena and Reggio Emilia          1
University of Glasgow                           1
Shanghai Jiao Tong University                   1
Name: count, Length: 109, dtype: int64

In [29]:
paper_auth_pairs_FH = paper_auth_pairs.drop(columns=['last author']).sort_values("first author", ascending=True)
paper_auth_pairs_FH

,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
243,Abdelsadik A,Male,Senior Staff,True,PI,False,Forschungszentrum Borstel,Germany,False,abdelsadik a,roeder t
220,Agaisse H,Male,Post-doc,True,PI,False,Harvard Medical School,USA,True,agaisse h,perrimon n
92,Agaisse H,Male,Post-doc,True,PI,False,Harvard Medical School,USA,True,agaisse h,higgins de
274,Aggarwal K,Female,PhD,False,Industry,False,University of Massachusetts Medical School,USA,False,aggarwal k,silverman n
315,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
...,...,...,...,...,...,...,...,...,...,...,...
218,Zhang Z,Female,PhD,False,Admin,False,University of Kentucky,USA,False,zhang z,palli sr
87,Zhao HW,Female,PhD,False,NaN,False,University of California San Diego,USA,True,zhao hw,haddad gg
141,Zhou Z,Male,PhD,NaN,NaN,False,University of Houston,USA,False,zhou z,kimbrell da
79,Zhuang ZH,Male,PhD,False,NaN,False,Shanghai Jiao Tong University,China,False,zhuang zh,ge bx


In [30]:
paper_auth_pairs_FH = preprocess_utils.deduplicate_by(paper_auth_pairs_FH, "first author")

            among ['perrimon n', 'higgins de'].
            among ['Post-doc', 'PhD'].
            among ['Harvard Medical School and Massachusetts General Hospital', 'Massachusetts General Hospital'].
            among ['rahme lg', 'mylonakis e'].
            among ['PhD', 'Post-doc'].
            among ['CEA Grenoble', 'Université de Toulouse'].
            among ['fauvarque mo', 'waltzer l'].
            among ['CNRS Gif-sur-Yvette Université Paris-Saclay', 'CNRS_Gif-sur-Yvette_Université Paris-Saclay'].
            among ['boccard f', 'lemaitre b'].
            among ['CNRS_Gif-sur-Yvette_Université Paris-Saclay', 'CNRS Gif-sur-Yvette Université Paris-Saclay'].
            among ['lemaitre b', 'brey pt'].
            among ['lagueux m', 'hoffmann ja'].
            among ['Massachusetts General Hospital', 'Harvard Medical School,Boston'].
            among ['white k', 'ezekowitz ra'].
            among ['Univeristy of Freiburg', 'Johannes Gutenberg Universität'].
            among [

In [ ]:

# check for duplicate first_author_key in paper_auth_pairs_LH
print(len(paper_auth_pairs_FH[paper_auth_pairs_FH.duplicated('first_author_key', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate first_author_key in leading_authors_claims
print(len(first_authors_claims[first_authors_claims.duplicated('first_author_key', keep=False)]), "duplicates in leading_authors_claims")


all_FH = pd.merge(first_authors_claims, paper_auth_pairs_FH, on='first_author_key', how='outer', suffixes=('', '_pairs'))
print(len(first_authors_claims), len(paper_auth_pairs_FH), len(all_FH))

print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
unique_pairs = all_FH[["Name", "first author", "first_author_key"]].drop_duplicates().sort_values("first_author_key", ascending=True)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['first author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f" {unique_pairs.iloc[i]['first author']:<20}  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['first_author_key']:<20}")

0 duplicates in paper_auth_pairs_LH
4 duplicates in leading_authors_claims
292 291 294
 ONLY in Pairs               only in Claims        matching key 
----------------------------------------------------------------------
💥  nan                   Hedengren-Olcott M   --> hedengren-olcott m  
💥  RIZKI MT              nan                  --> rizki mt            
💥  Schneider D           nan                  --> schneider d         


In [32]:


all_FH = all_FH.apply(preprocess_utils.safe_strip)

In [ ]:
all_FH_inner = pd.merge(first_authors_claims, paper_auth_pairs_FH, on='first_author_key', how='inner', suffixes=('', '_pairs')).sort_values("first_author_key", ascending=True)
for idx, row in all_FH_inner.iterrows():
    if row["Sex"] != row["Sex_pairs"]:
        print(f"confligcthing sex for {row['Name']} : {row['Sex']} vs  {row['Sex_pairs']}")
all_FH_inner.drop(columns=["Sex_pairs"], inplace=True)
print(len(all_FH_inner))
all_FH_inner = all_FH_inner.apply(preprocess_utils.safe_strip)

assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']
# set type of assessment columns to int
for col in assessment_columns:
    all_FH_inner[col] = all_FH_inner[col].astype(int)
all_FH_inner['Major claims'] = all_FH_inner['Major claims'].astype(int)
all_FH_inner['Articles'] = all_FH_inner['Articles'].astype(int)

for col in assessment_columns:
    all_FH_inner[f'{col}_prop'] = all_FH_inner[col] / all_FH_inner['Major claims']

all_FH_inner.to_csv('preprocessed_data/FH_inner.csv', index=False)
all_FH_inner

confligcthing sex for Ao J : Male vs  nan
confligcthing sex for Avila A : Male vs  Female
confligcthing sex for Cheng W : Female vs  nan
confligcthing sex for Chiu H : Male vs  Female
confligcthing sex for Daffre S : Male vs  Female
confligcthing sex for Ertürk-Hasdemir D : Male vs  Female
confligcthing sex for Han SH : Male vs  Female
confligcthing sex for Hedengren M : Male vs  Female
confligcthing sex for Kim YS : Female vs  Male
confligcthing sex for Kwon EJ : Male vs  Female
confligcthing sex for Lindmark H : Female vs  Male
confligcthing sex for Luce-Fedrow A : Female vs  Male
confligcthing sex for Michel T : Female vs  Male
confligcthing sex for Mishima Y : Female vs  Male
confligcthing sex for Nam HJ : Male vs  Female
confligcthing sex for Peng J : Male vs  Female
confligcthing sex for Persson C : Male vs  Female
confligcthing sex for Pham LN : Male vs  Female
confligcthing sex for Ren C : Female vs  Male
confligcthing sex for Schmidt RL : Female vs  Male
confligcthing sex for 

,Name,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Authorship,...,MD,Affiliation,Country,Ivy league,last_author_key,Unchallenged_prop,Verified_prop,Partially Verified_prop,Mixed_prop,Challenged_prop
79,Abdelsadik A,Male,1,3,3,0,0,0,0,First,...,False,Forschungszentrum Borstel,Germany,False,roeder t,1.000000,0.000000,0.000000,0.0,0.0
19,Agaisse H,Male,2,6,2,4,0,0,0,First,...,False,Harvard Medical School,USA,True,perrimon n,0.333333,0.666667,0.000000,0.0,0.0
195,Aggarwal K,Female,1,2,0,2,0,0,0,First,...,False,University of Massachusetts Medical School,USA,False,silverman n,0.000000,1.000000,0.000000,0.0,0.0
80,Ao J,Male,1,3,1,2,0,0,0,First,...,False,niversity of Missouri-Kansas City,USA,False,yu xq,0.333333,0.666667,0.000000,0.0,0.0
20,Apidianakis Y,Male,3,6,4,2,0,0,0,First,...,False,Harvard Medical School and Massachusetts Gener...,USA,True,rahme lg,0.666667,0.333333,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Zettervall CJ,Male,1,3,0,3,0,0,0,First,...,False,University of Stockholm,Sweden,False,hultmark d,0.000000,1.000000,0.000000,0.0,0.0
273,Zhang Z,Female,2,2,1,1,0,0,0,First,...,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhu s,0.500000,0.500000,0.000000,0.0,0.0
194,Zhao HW,Female,1,3,2,1,0,0,0,First,...,False,University of California San Diego,USA,True,haddad gg,0.666667,0.333333,0.000000,0.0,0.0
290,Zhou Z,Male,1,1,0,1,0,0,0,First,...,False,University of Houston,USA,False,kimbrell da,0.000000,1.000000,0.000000,0.0,0.0


In [34]:
for aut in authors[authors["first_author"]]["author_key"]:
    #if len(all_FH[all_LH["first_author_key"] == aut]) != 1:
    #    print (f"outer: 💥 {aut}, {len(all_FH[all_FH['author_key'] == aut])}")
    if len(all_FH_inner[all_FH_inner["first_author_key"] == aut]) != 1:
        print (f"inner: 💥 {aut}, {len(all_FH_inner[all_FH_inner['first_author_key'] == aut])}")

inner: 💥 hedengren-olcott m, 0
inner: 💥 hedengren m, 2
inner: 💥 hedengren m, 2
inner: 💥 kim ys, 2
inner: 💥 kim ys, 2
inner: 💥 kim ys, 2


In [35]:
all_FH_inner.columns

Index(['Name', 'Sex', 'Articles', 'Major claims', 'Unchallenged', 'Verified',
       'Partially Verified', 'Mixed', 'Challenged', 'Authorship',
       'first_author_key', 'first author', 'PhD Post-doc', 'Become a Pi',
       'current job', 'MD', 'Affiliation', 'Country', 'Ivy league',
       'last_author_key', 'Unchallenged_prop', 'Verified_prop',
       'Partially Verified_prop', 'Mixed_prop', 'Challenged_prop'],
      dtype='object')

In [ ]:
for col in all_FH_inner.columns:
    print(70*"-")
    print(all_FH_inner[col].value_counts())

## Summary
So how to analyze ? The key works well to merge at least, so let's have it. I think we should not do by authors because the aurhors are fleeting. Whether it's first or not. Let's do it by claim when it is needed. I think it's much more correct.